Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [4]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

print("SCRIPTS_PATH:", SCRIPTS_PATH)
print("ANNOTATION_PATH:", ANNOTATION_PATH)
print("IMAGE_PATH:", IMAGE_PATH)

SCRIPTS_PATH: Tensorflow/scripts
ANNOTATION_PATH: Tensorflow/workspace/annotations
IMAGE_PATH: Tensorflow/workspace/images


# 1. Create Label Map

In [5]:
import os

# Your label maps
labels = [
    {'name': 'Hello', 'id': 1}, 
    {'name': 'Yes', 'id': 2}, 
    {'name': 'No', 'id': 3}, 
    {'name': 'ILoveYou', 'id': 4}, 
    {'name': 'ThankYou', 'id': 5}
]

# Ensure the ANNOTATION_PATH directory exists
os.makedirs(ANNOTATION_PATH, exist_ok=True)

# Correctly join the paths to avoid escape sequence issues
file_path = os.path.join(ANNOTATION_PATH, 'label_map.pbtxt')

# Write the label map to the file
with open(file_path, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [3]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}


Python executable: c:\Users\jugra\anaconda3\python.exe
TensorFlow version: 2.19.0
Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Python executable: c:\Users\jugra\anaconda3\python.exe
TensorFlow version: 2.19.0
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
Updating files:  26% (1013/3893)
Updating files:  27% (1052/3893)
Updating files:  28% (1091/3893)
Updating files:  29% (1129/3893)
Updating files:  30% (1168/3893)
Updating files:  31% (1207/3893)
Updating files:  32% (1246/3893)
Updating files:  33% (1285/3893)
Updating files:  34% (1324/3893)
Updating files:  35% (1363/3893)
Updating files:  36% (1402/3893)
Updating files:  37% (1441/3893)
Updating files:  38% (1480/3893)
Updating files:  39% (1519/3893)
Updating files:  40% (1558/3893)
Updating files:  41% (1597/3893)
Updating files:  42% (1636/3893)
Updating files:  43% (1674/3893)
Updating files:  44% (1713/3893)
Updating files:  45% (1752/3893)
Updating files:  46% (1791/3893)
Updating files:  46% (1826/3893)
Updating files:  47% (1830/3893)
Updating files:  48% (1869/3893)
Updating files:  49% (1908/3893)
Updating files:  50% (1947/3893)
Updating files:  51% (1986/3893)
Updating files:  52% (2025/3893)
Updating files:  53% (2064/3893)
Updating files:  5

In [ ]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [6]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [7]:
# Define variables
CUSTOM_MODEL_NAME = "my_ssd_mobnet"
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
MODEL_PATH = "Tensorflow/workspace/models"

# Create directory (if it doesn't exist)
!mkdir "{MODEL_PATH}/{CUSTOM_MODEL_NAME}" 2> nul || echo Directory already exists or cannot be created

# Copy pipeline.config file
!copy "{PRETRAINED_MODEL_PATH}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config" "{MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config"

# Verify
!dir "{MODEL_PATH}/{CUSTOM_MODEL_NAME}"

Directory already exists or cannot be created
        0 file(s) copied.


The system cannot find the file specified.


 Volume in drive C is Local Disk
 Volume Serial Number is FE3C-B042

 Directory of c:\Users\jugra\Documents\GitHub\MajorProjectComp8800\Tensorflow\workspace\models\my_ssd_mobnet

2025-03-10  02:08 PM    <DIR>          .
2025-03-10  02:08 PM    <DIR>          ..
2025-03-12  04:03 PM             4,444 pipeline.config
               1 File(s)          4,444 bytes
               2 Dir(s)  52,911,280,128 bytes free


# 5. Update Config For Transfer Learning

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Define the path to pipeline.config file
CONFIG_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

# Load the configuration file
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Print the loaded configs to verify
print(configs)

{'model': ssd {
  num_classes: 5
  image_resizer {
    fixed_shape_resizer {
      height: 320
      width: 320
    }
  }
  feature_extractor {
    type: "ssd_mobilenet_v2_fpn_keras"
    depth_multiplier: 1.0
    min_depth: 16
    conv_hyperparams {
      regularizer {
        l2_regularizer {
          weight: 4e-05
        }
      }
      initializer {
        random_normal_initializer {
          mean: 0.0
          stddev: 0.01
        }
      }
      activation: RELU_6
      batch_norm {
        decay: 0.997
        scale: true
        epsilon: 0.001
      }
    }
    use_depthwise: true
    override_base_feature_extractor_hyperparams: true
    fpn {
      min_level: 3
      max_level: 7
      additional_layer_depth: 128
    }
  }
  box_coder {
    faster_rcnn_box_coder {
      y_scale: 10.0
      x_scale: 10.0
      height_scale: 5.0
      width_scale: 5.0
    }
  }
  matcher {
    argmax_matcher {
      matched_threshold: 0.5
      unmatched_threshold: 0.5
      ignore_threshold

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [10]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [11]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
# This is the command I need to run 

print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [14]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'estimator'

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [ ]:
import cv2 
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
cap.release()

In [ ]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt